In [8]:
## Import libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV 

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, r2_score

pd.options.display.float_format = '{:,.2f}'.format
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)



In [9]:
## Read data
df = pd.read_csv('../Data/final_inflation_adj_data.csv')


In [10]:
## Filter data from after the Netflix launch and before 2020 to prevent the outliers

df = df[(df['year']>=2007)&(df['year']<2020)]

In [11]:
df.columns

Index(['name', 'rating', 'genre', 'year', 'score', 'votes', 'director',
       'writer', 'star', 'country', 'budget', 'gross', 'company', 'runtime',
       'month', 'rating_imputed', 'budget_imputed', 'inf_adjmnt_value',
       'budget_adj', 'gross_adj', 'profit', 'profit_adj'],
      dtype='object')

In [12]:
# Define the features and the target variable

X = df.drop(['name', 'gross', 'profit', 'profit_adj', 'gross_adj', 'rating_imputed', 'budget_imputed','inf_adjmnt_value'], axis=1)
y = df['profit_adj']


In [13]:
# Define numerical and categorical columns

numerical_cols = ['rating', 'runtime', 'budget_adj']## ,'score', 'votes',
categorical_cols = ['genre', 'company', 'month'] ##'director', 'writer', 'star', 'country', 


In [14]:
X=df[categorical_cols + numerical_cols]
y= df['profit_adj']

In [15]:

#instantiae OnehotEncoder
ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = True)

#preprocessing steps
ss = StandardScaler()
ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = False)
ctx = ColumnTransformer(
                            transformers = [
                                 ('one_hot', ohe, categorical_cols ),
                                 ('sc',  ss,   numerical_cols)
                                ] 
                                   , verbose_feature_names_out=False
                                    , remainder='passthrough'
                        
                            ) 


In [16]:
#split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state =42)
X_train.shape, X_test.shape


((1941, 6), (648, 6))

In [17]:
## fit and transform training data

X_train = ctx.fit_transform(X_train)
X_train =pd.DataFrame(X_train, columns=ctx.get_feature_names_out())
X_train.shape

(1941, 799)

In [18]:
##transform test data

X_test = ctx.transform(X_test)
X_test =pd.DataFrame(X_test, columns=ctx.get_feature_names_out())
X_test.shape

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


(648, 799)

In [19]:
X_test.head()

,genre_Adventure,genre_Animation,genre_Biography,genre_Comedy,genre_Crime,genre_Drama,genre_Family,genre_Fantasy,genre_Horror,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Sport,genre_Thriller,company_120dB Films,company_1984 Private Defense Contractors,company_2.0 Entertainment,company_2.4.7. Films,company_21 Laps Entertainment,company_26 Films,company_2929 Productions,company_2DUX²,company_3 Arts Entertainment,company_3311 Productions,company_360 Pictures,company_4Kids Entertainment,company_7DM Studios,company_8383 Productions,company_A Lucky Old Sun Production,company_A24,company_ABS-CBN Film Productions,company_ADHM Films,company_AI-Film,company_AOI Promotion,company_AR Films,company_ASIG Productions,company_ATO Pictures,company_Aamir Khan Productions,company_Abandon Entertainment,company_Abbolita Productions,company_About Film,company_Abraham Productions,company_Acacia Filmed Entertainment,company_Access Entertainment,company_Ace Maker Movie Works,company_Ada Films,company_Affirm Films,company_After Dark Films,company_Aggregate Films,company_Agile Films,company_Alacran Pictures,company_Alameda Films,company_Alcatraz Films,company_Alcon Entertainment,company_Alcove Entertainment,company_Aldamisa Entertainment,company_Alibaba Pictures,company_Alliance,company_Alliance Films,company_Allison Shearmur Productions,company_Alloy Entertainment,company_Allspark Pictures,company_Also Known As Pictures,company_Altar Identity Studios,company_Amasia Entertainment,company_Amazon Studios,company_Amber Entertainment,company_Ambience Entertainment,company_Amblin Entertainment,company_American Zoetrope,company_Amuse Soft,company_Amusement Park Films,company_Anchor Bay Films,company_Andrew Lauren Productions,company_Angry Films,company_Animal Kingdom,company_Animal Logic,company_Anna Biller Productions,company_Annapurna Pictures,company_Annapurna Productions,company_Anonymous Content,company_Antena 3 Films,company_Apartment Story,company_Apartment Two,company_Apatow Productions,company_Apeitda,company_Appian Way,company_Aquarius Films,company_Aramid Entertainment Fund,company_Archer Gray,company_Archimede,company_ArieScope Pictures,company_Aristar Entertainment,company_Arka Mediaworks,company_Armory Films,company_Artbees,company_Artists First,company_Ascendant Pictures,company_Astrakan Films,company_Astute Films,company_Atlantic Pictures (II),company_Atlantic Records,company_Atlas Entertainment,company_Atlas Productions,company_Atomic Monster,company_Audax Films,company_Audiovisual Aval SGR,company_Aurora Alliance Films,company_Automatik Entertainment,company_Avi Arad & Associates,company_Avi Arad Productions,company_Awesomeness Films,company_B Story,company_B24,company_BBC Films,company_BCDF Pictures,company_BET Films,company_BFI Film Fund,company_BRON Studios,company_BZ Entertainment,company_Babylon,company_Bad Robot,company_Bake My Cake Films,company_Balthazar Productions,company_Bank Street Films,company_Barbarian Films,company_Barunson E&A,company_Bazelevs Production,company_Beachside Films,company_Becker Films,company_Before The Door Pictures,company_Beijing Dengfeng International Culture Communications Company,company_Beijing Film Studio,company_Beijing New Picture Film,company_Bellanova Films,company_Bellwether Pictures,company_BenderSpink,company_Berk/Lane Entertainment,company_Bhansali Productions,company_Big Beach Films,company_Big City Pictures,company_Big Indie Pictures,company_Big Screen Productions,company_Big Talk Productions,company_Bill Kenwright Films,company_Black Bear Pictures,company_Black Bicycle Entertainment,company_Black Entertainment Television (BET),company_Black Label Media,company_BlackWhite,company_Blank of the Dead Productions,company_Bleecker Street Films,company_Blinding Edge Pictures,company_Block 2 Pictures,company_Bloom,company_Blue Askew,company_Blue Budgie Films Limited,company_Blue Haze Entertainment,company_Blue Sky Films,company_Blue Sky Studios,company_Blue Yonder Films,company_Bluegrass Films,com

In [20]:
#Lists to hold result metrices

models=[]
Train_r2 = []
Test_r2 =[]
RMSE =[]
CV_score =[]

#### Ordinary Linear Regression

In [21]:
#Instantiate Grid search pipeline for Ordinary Linear Regression

pipe_lr = Pipeline([('lr', LinearRegression())]
                   )
params = {}
gs_lr = GridSearchCV(pipe_lr, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_lr.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('lr', LinearRegression())]),
             n_jobs=-1, param_grid={}, scoring='r2')

In [24]:
#Evaluate the  model 
y_pred_lr = gs_lr.best_estimator_.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)


print(f'Train rsquare: {gs_lr.score(X_train, y_train)}')
print(f'Test rsquare: {gs_lr.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_lr)}')
print(f'Best CV score: {gs_lr.best_score_}')
print(f'Best Parameters: {gs_lr.best_params_}')

models.append('Linear Regression')
Train_r2.append(gs_lr.score(X_train, y_train))
Test_r2.append(gs_lr.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_lr))
CV_score.append(gs_lr.best_score_)

Train rsquare: 0.6020664064552794
Test rsquare: -5.277424012104136e+19
RMSE: 1.5478667384746102e+18
Best CV score: -2.232831750211942e+24
Best Parameters: {}


#### Ridge Regression

In [25]:
#Instantiate Grid search pipeline for Ridge Regression

pipe_rd = Pipeline([('rd', Ridge())]
                   )
params = {'rd__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

gs_rd = GridSearchCV(pipe_rd, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_rd.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('rd', Ridge())]), n_jobs=-1,
             param_grid={'rd__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='r2')

In [26]:
# # Evaluate the  model on the test set
y_pred_rd = gs_rd.best_estimator_.predict(X_test)
r2 = r2_score(y_test, y_pred_rd)

print(f'Train rsquare: {gs_rd.score(X_train, y_train)}')
print(f'Test rsquare: {gs_rd.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_rd)}')
print(f'Best CV score: {gs_rd.best_score_}')
print(f'Best Parameters: {gs_rd.best_params_}')

models.append('Ridge')
Train_r2.append(gs_rd.score(X_train, y_train))
Test_r2.append(gs_rd.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_rd))
CV_score.append(gs_rd.best_score_)

Train rsquare: 0.51697096390494
Test rsquare: 0.4521767949727916
RMSE: 157704027.916039
Best CV score: 0.4467970109212934
Best Parameters: {'rd__alpha': 10.0}


#### LASSO Regression

In [27]:
#Instantiate Grid search pipeline for Lasso Regression

pipe_ls = Pipeline([('ls', Lasso())]
                   )
params = {'ls__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

gs_ls = GridSearchCV(pipe_ls, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_ls.fit(X_train, y_train)

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.519e+19, tolerance: 7.836e+15
  model = cd_fast.enet_coordinate_descent(
/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.519e+19, tolerance: 7.836e+15
  model = cd_fast.enet_coordinate_descent(
/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

GridSearchCV(cv=5, estimator=Pipeline(steps=[('ls', Lasso())]), n_jobs=-1,
             param_grid={'ls__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]},
             scoring='r2')

In [28]:
y_pred_ls = gs_ls.best_estimator_.predict(X_test)

print(f'Train rsquare: {gs_ls.score(X_train, y_train)}')
print(f'Test rsquare: {gs_ls.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_ls)}')
print(f'Best CV score: {gs_ls.best_score_}')
print(f'Best Parameters: {gs_ls.best_params_}')

models.append('Lasso')
Train_r2.append(gs_ls.score(X_train, y_train))
Test_r2.append(gs_ls.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_ls))
CV_score.append(gs_ls.best_score_)

Train rsquare: 0.6021045747187834
Test rsquare: 0.322263397066856
RMSE: 175409473.22034398
Best CV score: 0.3870470635942472
Best Parameters: {'ls__alpha': 100.0}


#### Random Forest Regressor

In [29]:
#Instantiate Grid search pipeline for Random Forest Regression

pipe_rf = Pipeline([('rf', RandomForestRegressor())]
                   )
params = {
            'rf__n_estimators': [50, 100, 200],
            'rf__max_features': ['auto', 'sqrt', 'log2']
         }

gs_rf = GridSearchCV(pipe_rf, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_rf.fit(X_train, y_train)

/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
15 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/pujak.c./anaconda3/lib/python3.10/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt

GridSearchCV(cv=5, estimator=Pipeline(steps=[('rf', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'rf__max_features': ['auto', 'sqrt', 'log2'],
                         'rf__n_estimators': [50, 100, 200]},
             scoring='r2')

In [30]:
y_pred_rf = gs_rf.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_rf.score(X_train, y_train)}')
print(f'Test rsquare: {gs_rf.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_rf)}')
print(f'Best CV score: {gs_rf.best_score_}')
print(f'Best Parameters: {gs_rf.best_params_}')



models.append('Random Forest')
Train_r2.append(gs_rf.score(X_train, y_train))
Test_r2.append(gs_rf.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_rf))
CV_score.append(gs_rf.best_score_)

Train rsquare: 0.939275736708425
Test rsquare: 0.4844621727935241
RMSE: 152986405.93049088
Best CV score: 0.4876052348666874
Best Parameters: {'rf__max_features': 'sqrt', 'rf__n_estimators': 100}


#### AdaBoost Regressor

In [31]:
#Instantiate Grid search pipeline for Ada Boost Regressor

pipe_ada = Pipeline([('ada', AdaBoostRegressor())]
                   )
params = {
            'ada__n_estimators': [50, 100, 200],
            'ada__learning_rate': [0.01, 0.1, 0.2]
        }

gs_ada = GridSearchCV(pipe_ada, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_ada.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('ada', AdaBoostRegressor())]),
             n_jobs=-1,
             param_grid={'ada__learning_rate': [0.01, 0.1, 0.2],
                         'ada__n_estimators': [50, 100, 200]},
             scoring='r2')

In [32]:
y_pred_ada = gs_ada.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_ada.score(X_train, y_train)}')
print(f'Test rsquare: {gs_ada.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_ada)}')
print(f'Best CV score: {gs_ada.best_score_}')
print(f'Best Parameters: {gs_ada.best_params_}')

models.append('Ada Boost')
Train_r2.append(gs_ada.score(X_train, y_train))
Test_r2.append(gs_ada.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_ada))
CV_score.append(gs_ada.best_score_)

Train rsquare: 0.6005976196768803
Test rsquare: 0.3836324222584355
RMSE: 167279401.69629934
Best CV score: 0.4501064371898399
Best Parameters: {'ada__learning_rate': 0.01, 'ada__n_estimators': 100}


#### GradientBoosting Regressor

In [33]:
#Instantiate Grid search pipeline for Gradient Boosting Regressor

pipe_gbr = Pipeline([('gbr', GradientBoostingRegressor())]
                   )
params = {
            'gbr__n_estimators': [50, 100, 200],
            'gbr__learning_rate': [0.01, 0.1, 0.2],
            'gbr__max_depth': [3, 5, 7]
     
        }

gs_gbr = GridSearchCV(pipe_gbr, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
gs_gbr.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('gbr', GradientBoostingRegressor())]),
             n_jobs=-1,
             param_grid={'gbr__learning_rate': [0.01, 0.1, 0.2],
                         'gbr__max_depth': [3, 5, 7],
                         'gbr__n_estimators': [50, 100, 200]},
             scoring='r2')

In [34]:
y_pred_gbr = gs_gbr.best_estimator_.predict(X_test)


print(f'Train rsquare: {gs_gbr.score(X_train, y_train)}')
print(f'Test rsquare: {gs_gbr.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_gbr)}')
print(f'Best CV score: {gs_gbr.best_score_}')
print(f'Best Parameters: {gs_gbr.best_params_}')

models.append('GradientBoost')
Train_r2.append(gs_gbr.score(X_train, y_train))
Test_r2.append(gs_gbr.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_gbr))
CV_score.append(gs_gbr.best_score_)


Train rsquare: 0.7001826169206896
Test rsquare: 0.4044538724057122
RMSE: 164429704.42461902
Best CV score: 0.4679992935182785
Best Parameters: {'gbr__learning_rate': 0.1, 'gbr__max_depth': 3, 'gbr__n_estimators': 50}


#### Bagging Regressor

In [35]:
#Instantiate Grid search pipeline for Bagging Regressor

# Initialize the base estimator
bg = DecisionTreeRegressor()

# Initialize a Bagging Regressor with the base decision tree regressor
bagging_regressor = BaggingRegressor(estimator=bg, random_state=42, n_jobs=-1)

# Define the parameter grid
param_grid = {
    'n_estimators': [10, 50, 100],
    'estimator__max_depth': [None, 10, 20, 30],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV with the Bagging Regressor
gs_br = GridSearchCV(estimator=bagging_regressor, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1, verbose=2)

# Train the GridSearchCV
gs_br.fit(X_train, y_train)

# Make predictions on the test set with the best model
best_model = gs_br.best_estimator_
predictions = best_model.predict(X_test)


Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [36]:
y_pred_br = gs_br.best_estimator_.predict(X_test)

print(f'Train rsquare: {gs_br.score(X_train, y_train)}')
print(f'Test rsquare: {gs_br.score(X_test, y_test)}')
print(f'RMSE: {root_mean_squared_error(y_test, y_pred_br)}')
print(f'Best CV score: {gs_br.best_score_}')
print(f'Best Parameters: {gs_br.best_params_}') 

models.append('Bagging Regressor')
Train_r2.append(gs_br.score(X_train, y_train))
Test_r2.append(gs_br.score(X_test, y_test))
RMSE.append(root_mean_squared_error(y_test, y_pred_br))
CV_score.append(gs_br.best_score_)

Train rsquare: -1.4904128763415002e+16
Test rsquare: -2.7712147020014096e+16
RMSE: 166469657.95607948
Best CV score: -2.5082155792704576e+16
Best Parameters: {'estimator__max_depth': 10, 'estimator__min_samples_leaf': 4, 'estimator__min_samples_split': 10, 'n_estimators': 10}


In [37]:
Train_r2 = [round(num, 4) for num in Train_r2]
Test_r2 = [round(num, 4) for num in Test_r2]
RMSE = [f'{num:,.2f}' for num in RMSE]  # Format with comma and 2 decimal places
CV_score = [round(num, 4) for num in CV_score]

In [38]:
# Create DataFrame
data = {'Model': models, 'Train_r2': Train_r2, 'Test_r2': Test_r2, 'RMSE': RMSE, 'CV_score': CV_score}
result = pd.DataFrame(data)


In [40]:
result

,Model,Train_r2,Test_r2,RMSE,CV_score
0,Linear Regression,0.60,"-52,774,240,121,041,362,944.00","1,547,866,738,474,610,176.00","-2,232,831,750,211,941,903,106,048.00"
1,Ridge,0.52,0.45,"157,704,027.92",0.45
2,Lasso,0.60,0.32,"175,409,473.22",0.39
3,Random Forest,0.94,0.48,"152,986,405.93",0.49
4,Ada Boost,0.60,0.38,"167,279,401.70",0.45
5,GradientBoost,0.70,0.40,"164,429,704.42",0.47
6,Bagging Regressor,"-14,904,128,763,415,004.00","-27,712,147,020,014,096.00","166,469,657.96","-25,082,155,792,704,576.00"


#### From above model evaluation Random Forest Regressor has least RMSE and Training Rsquare value of 0.94 and Testing Rsquare value of 0.48 also it has maximum cross val score of .49 and Gradient Boost Regressor seems to outperform remainig other modes.  We will do further feature engineering and hyper tuning to evaluate these two models to get ourt best performing model.